# Implementación

In [1]:
# Comprueba que la molecula es completa
def IsComplete(molecule, isComplementary):
  # Comprobamos longitud
  if len(molecule[0]) != len(molecule[1]):
    return False
  # Comprobamos que cumple la relación de complementariedad
  for i, _ in enumerate(molecule[0]):
    if not isComplementary(molecule[0][i],molecule[1][i]):
      return False
  return True

In [2]:
# Comprueba que todas las letras de la molecula pertenecen a V
def IsFromVocabulary(molecule, V):
  for i, _ in enumerate(molecule[0]):
    if molecule[0][i] not in V:
      return False
    if molecule[1][i] not in V:
      return False
  return True

In [3]:
# Comprueba que la molecula contiene el axioma
def ContainsAxiom(molecule, A):
  if A[0] not in molecule[0]:
    return False
  if A[1] not in molecule[1]:
    return False
  return True

In [4]:
# Comprueba que el sistema de stickers es regular
def IsRegular(D):
  for d in D:
    if d['l'][0] != "" or d['l'][1] != "":
      return False
  return False

In [5]:
def Apply(m, d):
  dl0 = d['l'][0]
  dl1 = d['l'][1]
  dr0 = d['r'][0]
  dr1 = d['r'][1]
  m[0] = dl0 + m[0] + dr0
  m[1] = dl1 + m[1] + dr1

def Revert(m, d):
  dl0 = d['l'][0]
  dl1 = d['l'][1]
  dr0 = d['r'][0]
  dr1 = d['r'][1]
  if m[0].startswith(dl0) and m[1].startswith(dl1) and m[0].endswith(dr0) and m[1].endswith(dr1):
    if len(dl0) > 0: m[0] = m[0][len(dl0):]
    if len(dl1) > 0: m[1] = m[1][len(dl1):]
    if len(dr0) > 0: m[0] = m[0][:-len(dr0)]
    if len(dr1) > 0: m[1] = m[1][:-len(dr1)]
    return True
  return False

In [6]:
def TryRevert(p, m, di, D_applied):
  A = p["A"]
  D = p["D"]
  #print(f"Trying {str(D_applied + [di])}")
  successful = Revert(m, D[di])
  if successful:
    #print(f"Reverted m:{str(m)} using d{di}:{str(D[di])}")
    D_applied.append(di)
    di = 0
    if m[0] is A[0] and m[1] is A[1]:
      return 0, di, D_applied, "Hemos llegado al axioma"
    if len(m[0]) == len(A[0]) or len(m[1]) == len(A[1]):
      return OnFail(p, m, di, D_applied) # Hemos llegado a un posible axioma incorrecto
    else:
      return 2, di, D_applied, "Continuar"
  else:
    return OnFail(p, m, di, D_applied) # El domino elegido no se puede usar para revertir 

def OnFail(p, m, di, D_applied):
  D = p["D"]
  di += 1
  if di >= len(D): # Se han probado todos los dominos para este nivel
    if len(D_applied) == 0:
      return 1, di, D_applied, "Todos los dominos probados son invalidos"
    else:
      di = D_applied.pop()
      Apply(m, D[di])
      return OnFail(p, m, di, D_applied)
  else:
    return 2, di, D_applied, "Continuar"

In [7]:
def AreDominoesCorrect(p, m0, D_applied):
  D = p["D"]
  A = p["A"]
  m = list(m0)
  print(m)
  for di in D_applied:
    successful = Revert(m,D[di])
    if not successful:
      return False, f"No se puede revertir {m} usando {D[di]}"
    else:
      print(f"Revertido a {m} usando {D[di]}")
  if m[0] is A[0] and m[1] is A[1]:
    return True, "Hemos llegado al axioma"
  else:
    return False, "Hemos llegado a un resultado incorrecto"

In [8]:
def CanGenerate(p, m0, maxIterations):
  D_applied = []
  m = list(m0)
  if not IsComplete(m, p["Y"]):
    return "La molécula " + str(m0) + " no es completa"
  if not IsFromVocabulary(m, p["V"]):
    return "La molécula " + str(m0) + " contiene caracteres que no están en el vocabulario"
  if not ContainsAxiom(m, p["A"]):
    return "La molécula " + str(m0) + " no contiene al axioma"

  di = 0
  for i in range(maxIterations):
    result, di, D_applied, msg = TryRevert(p, m, di, D_applied)
    if result == 1:
      return "El sistema no puede generar la molécula " + str(m0)
    if result == 0:
      return "El sistema genera la molécula " + str(m0) + " a partir del axioma " + str(p["A"]) + " aplicando los dominoes " + str([p["D"][d_idx] for d_idx in list(reversed(D_applied))])
  return "El sistema no puede generar la molécula " + str(m0)

# Sistemas de stickers no regulares

In [9]:
def isComplementary(a,b):
  if a == b:
    return True
  return False

In [10]:
V1 = {'a','b'}

d10 = { 'l': ['a','a'], 'r': ['a','a']}
d11 = { 'l': ['b','b'], 'r': ['b','b']}
D1 = ( d10, d11 )

A1 = ['a','a']

p_no_regular_1 = { 'V':V1, 'A':A1, 'Y':isComplementary, 'D':D1 }

In [11]:
V2 = {'a','b','c'}

d20 = { 'l': ['b',''], 'r': ['b','']}
d21 = { 'l': ['c',''], 'r': ['','']}
d22 = { 'l': ['','b'], 'r': ['','']}
d23 = { 'l': ['','c'], 'r': ['','b']}
D2 = ( d20, d21, d22, d23 )

A2 = ['a','a']

p_no_regular_2 = { 'V':V2, 'A':A2, 'Y':isComplementary, 'D':D2 }

In [12]:
V3 = {'a','b','c'}

d30 = { 'l': ['a',''], 'r': ['b','']}
d31 = { 'l': ['',''], 'r': ['c','']}
d32 = { 'l': ['',''], 'r': ['','b']}
d33 = { 'l': ['','a'], 'r': ['','c']}
D3 = ( d30, d31, d32, d33 )

A3 = ['a','a']

p_no_regular_3 = { 'V':V3, 'A':A3, 'Y':isComplementary, 'D':D3 }

In [13]:
w = "babbabbab"
m0 = [w,w]
m = list(m0)

print("SISTEMA DE STICKERS NO REGULAR p1")
print(CanGenerate(p_no_regular_1, m0, 100000))

w = "ccbbabb"
m0 = [str(w),str(w)]
m = list(m0)

print("SISTEMA DE STICKERS NO REGULAR p2")
print(CanGenerate(p_no_regular_2, m0, 100000))

w = "aaabbcc"
m0 = [str(w),str(w)]
m = m0

print("SISTEMA DE STICKERS NO REGULAR p3")
print(CanGenerate(p_no_regular_3, m0, 100000))

SISTEMA DE STICKERS NO REGULAR p1
El sistema genera la molécula ['babbabbab', 'babbabbab'] a partir del axioma ['a', 'a'] aplicando los dominoes [{'l': ['b', 'b'], 'r': ['b', 'b']}, {'l': ['b', 'b'], 'r': ['b', 'b']}, {'l': ['a', 'a'], 'r': ['a', 'a']}, {'l': ['b', 'b'], 'r': ['b', 'b']}]
SISTEMA DE STICKERS NO REGULAR p2
El sistema genera la molécula ['ccbbabb', 'ccbbabb'] a partir del axioma ['a', 'a'] aplicando los dominoes [{'l': ['', 'b'], 'r': ['', '']}, {'l': ['', 'b'], 'r': ['', '']}, {'l': ['', 'c'], 'r': ['', 'b']}, {'l': ['', 'c'], 'r': ['', 'b']}, {'l': ['b', ''], 'r': ['b', '']}, {'l': ['b', ''], 'r': ['b', '']}, {'l': ['c', ''], 'r': ['', '']}, {'l': ['c', ''], 'r': ['', '']}]
SISTEMA DE STICKERS NO REGULAR p3
El sistema genera la molécula ['aaabbcc', 'aaabbcc'] a partir del axioma ['a', 'a'] aplicando los dominoes [{'l': ['', ''], 'r': ['', 'b']}, {'l': ['', ''], 'r': ['', 'b']}, {'l': ['', 'a'], 'r': ['', 'c']}, {'l': ['', 'a'], 'r': ['', 'c']}, {'l': ['a', ''], 'r': ['

# Sistemas de stickers regulares

In [14]:
def isComplementary(a,b):
  if a == b:
    return True
  return False

In [15]:
V1 = {'a','b'}

d10 = { 'l': ['',''], 'r': ['a','a']}
d11 = { 'l': ['',''], 'r': ['b','b']}
D1 = ( d10, d11 )

A1 = ['a','a']

p_regular_1 = { 'V':V1, 'A':A1, 'Y':isComplementary, 'D':D1 }

In [16]:
V2 = {'a','b','c'}

d20 = { 'l': ['',''], 'r': ['b','']}
d21 = { 'l': ['',''], 'r': ['c','']}
d22 = { 'l': ['',''], 'r': ['','b']}
d23 = { 'l': ['',''], 'r': ['','c']}
D2 = ( d20, d21, d22, d23 )

A2 = ['a','a']

p_regular_2 = { 'V':V2, 'A':A2, 'Y':isComplementary, 'D':D2 }

In [17]:
w1 = "aaabbb"
w2 = "abbccc"
w3 = "abbbbb"
w4 = "baaaaa"

m1 = [w1,w1]
m2 = [w2,w2]
m3 = [w3,w3]
m4 = [w4,w4]

print("SISTEMA DE STICKERS REGULAR p1")
print(CanGenerate(p_regular_1, m1, 10000))
print(CanGenerate(p_regular_1, m2, 10000))
print(CanGenerate(p_regular_1, m3, 10000))
print(CanGenerate(p_regular_1, m4, 10000))
print('\n')

m1 = [w1,w1]
m2 = [w2,w2]
m3 = [w3,w3]
m4 = [w4,w4]

print("SISTEMA DE STICKERS REGULAR p2")
print(CanGenerate(p_regular_2, m1, 10000))
print(CanGenerate(p_regular_2, m2, 10000))
print(CanGenerate(p_regular_2, m3, 10000))
print(CanGenerate(p_regular_2, m4, 10000))
print('\n')

SISTEMA DE STICKERS REGULAR p1
El sistema genera la molécula ['aaabbb', 'aaabbb'] a partir del axioma ['a', 'a'] aplicando los dominoes [{'l': ['', ''], 'r': ['a', 'a']}, {'l': ['', ''], 'r': ['a', 'a']}, {'l': ['', ''], 'r': ['b', 'b']}, {'l': ['', ''], 'r': ['b', 'b']}, {'l': ['', ''], 'r': ['b', 'b']}]
La molécula ['abbccc', 'abbccc'] contiene caracteres que no están en el vocabulario
El sistema genera la molécula ['abbbbb', 'abbbbb'] a partir del axioma ['a', 'a'] aplicando los dominoes [{'l': ['', ''], 'r': ['b', 'b']}, {'l': ['', ''], 'r': ['b', 'b']}, {'l': ['', ''], 'r': ['b', 'b']}, {'l': ['', ''], 'r': ['b', 'b']}, {'l': ['', ''], 'r': ['b', 'b']}]
El sistema no puede generar la molécula ['baaaaa', 'baaaaa']


SISTEMA DE STICKERS REGULAR p2
El sistema no puede generar la molécula ['aaabbb', 'aaabbb']
El sistema genera la molécula ['abbccc', 'abbccc'] a partir del axioma ['a', 'a'] aplicando los dominoes [{'l': ['', ''], 'r': ['', 'b']}, {'l': ['', ''], 'r': ['', 'b']}, {'l': 